In [32]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath("../")

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, LeakyReLU, Conv2DTranspose, Conv2D, Reshape
from tensorflow.keras.activations import tanh

from aerosandbox import Airfoil, KulfanAirfoil
from src import airfoil_modifications

import plotly.graph_objects as go

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
class CSTGenerator(tf.keras.Model):
    '''
    * Refs: 
        - Lin, Jinxing & Zhang, Chenliang & Xie, Xiaoye & Shi, Xingyu & Xu, Xiaoyu & Duan, Yanhui. (2022). CST-GANs: A Generative Adversarial Network Based on CST Parameterization for the Generation of Smooth Airfoils. 600-605. 10.1109/ICUS55513.2022.9987080. 
    '''
    def __init__(self, npv: int = 12, latent_dim: int = 128, kernel_size: tuple = (2,4)):
        super().__init__()

        '''
        * npv: Number of parameterized variables
        * latent_dim: Dimension of input vector (latent vector)
        '''

        self.npv = npv
        self.latent_dim = latent_dim
        self.kernel_size = kernel_size

        # Fully connected layers
        self.dense1 = Dense(256 * 2 * self.npv, activation=LeakyReLU(0.2), input_shape=(self.latent_dim,))
        self.reshape = Reshape((2, self.npv, 256))

        # Transposed convolutions
        self.deconv1 = Conv2DTranspose(128, self.kernel_size, strides=(1,2), padding='same', activation=LeakyReLU(0.2))
        self.deconv2 = Conv2DTranspose(64, self.kernel_size, strides=(1,2), padding='same', activation=LeakyReLU(0.2))
        self.deconv3 = Conv2DTranspose(32, self.kernel_size, strides=(1,2), padding='same', activation=LeakyReLU(0.2))

        # Convolutional layers
        self.conv1 = Conv2D(16, self.kernel_size, strides=(1,2), padding="same", activation=LeakyReLU(0.2))
        self.conv2 = Conv2D(8, self.kernel_size, strides=(1,2), padding="same", activation=LeakyReLU(0.2))
        self.conv3 = Conv2D(1, self.kernel_size, strides=(1,2), padding="same", activation=tanh)

        # Output layers
        self.final_reshape = Reshape((2, self.npv))

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.reshape(x)
        x = self.deconv1(x)
        x = self.deconv2(x)
        x = self.deconv3(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.final_reshape(x)
        return x

model = CSTGenerator(kernel_size=(2,4))

/home/matsouto/Code/Py/airfoil_cst_gan/.venv/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [51]:
fig = go.Figure()
airfoil = Airfoil(name="NACA4412")
airfoil.draw(show=False, fig=fig, fill=False)

# Generate a random latent vector as input
latent_vector = np.random.randn(1, 128).astype(np.float32)
output = model(latent_vector)
print(output)

random_kulfan = KulfanAirfoil(lower_weights=output[0][0], upper_weights=output[0][1], leading_edge_weight=0, TE_thickness=0, N1=0.5, N2=1)
random_kulfan.draw(fig=fig, show=True, fill=False, color="green")

tf.Tensor(
[[[ 0.00703943 -0.00124994  0.00339497 -0.01190137  0.0059666
    0.00038682 -0.00848021 -0.00562707  0.00282079  0.00070263
    0.00569891 -0.00513325]
  [ 0.0009889  -0.00096795 -0.00214469 -0.00055222 -0.00731401
   -0.00326171  0.00188826 -0.00126161 -0.00622083  0.00137599
   -0.00690443  0.00297293]]], shape=(1, 2, 12), dtype=float32)
